<h1>gcForest Algorithm</h1>

<p>The gcForest algorithm was suggested in Zhou and Feng 2017 ( https://arxiv.org/abs/1702.08835 , refer for this paper for technical details) and I provide here a python3 implementation of this algorithm.<br>
I chose to adopt the scikit-learn syntax for ease of use and hereafter I present how it can be used.</p>

In [1]:
from GCForest import gcForest
from sklearn.datasets import load_iris, load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

<h2>Iris example</h2>

<p>The iris data set is actually not a very good example as the gcForest algorithm is better suited for time series and images where informations can be found at different scales in one sample.<br>
Nonetheless it is still an easy way to test the method.</p>

In [2]:
# loading the data
iris = load_iris()
X = iris.data
y = iris.target
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.33)

<p>First calling and training the algorithm.
A specificity here is the presence of the 'shape_1X' keyword to specify the shape of a single sample.
I have added it as pictures fed to the machinery might not be square.<br>
Obviously it is not very relevant for the iris data set but still, it has to be defined.</p>

In [3]:
gcf = gcForest(shape_1X=[4,1], window=[2], tolerance=0.0)
gcf.fit(X_tr, y_tr)

Slicing Sequence...
Training MGS Random Forests...
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.95
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.95


<p>Now checking the prediction for the test set:<p>

In [4]:
pred_X = gcf.predict(X_te)
print(pred_X)

Slicing Sequence...
[2 1 1 1 2 2 0 0 2 1 0 2 2 1 2 1 2 1 0 0 1 1 0 1 0 1 2 1 2 0 2 0 2 2 2 2 2
 2 1 0 1 2 1 1 1 1 1 1 0 2]


In [5]:
# evaluating accuracy
accuracy = accuracy_score(y_true=y_te, y_pred=pred_X)
print('gcForest accuracy : {}'.format(accuracy))

gcForest accuracy : 0.96


<h2>Digits Example</h2>
<p>A much better example is the digits data set containing images of hand written digits.
The scikit data set can be viewed as a mini-MNIST for training purpose.</p>

In [6]:
# loading the data
digits = load_digits()
X = digits.data
y = digits.target
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.4)

<p> ... taining gcForest ... (can take some time...) </p>

In [7]:
gcf = gcForest(shape_1X=[8,8], window=[4,6], tolerance=0.0, min_samples=7)
gcf.fit(X_tr, y_tr)

Slicing Images...
Training MGS Random Forests...
Slicing Images...
Training MGS Random Forests...
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.9814814814814815
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.9814814814814815


<p> ... and predicting classes ... </p>

In [8]:
pred_X = gcf.predict(X_te)
print(pred_X)

Slicing Images...
Slicing Images...
[2 3 3 7 6 2 8 1 9 2 5 5 6 9 7 9 8 0 9 0 7 0 5 2 2 7 1 6 5 5 5 2 7 4 9 4 0
 1 5 9 3 8 4 5 9 6 5 9 5 0 0 1 3 0 7 0 7 5 5 6 6 8 7 7 9 2 2 9 2 2 1 1 5 3
 2 4 0 4 9 1 3 5 2 6 0 9 6 3 3 7 3 6 0 4 8 8 8 2 2 5 4 7 5 2 0 1 6 6 5 9 6
 8 8 8 2 2 2 9 5 7 1 1 0 1 8 7 9 2 0 4 1 0 0 5 0 3 0 8 9 2 1 6 7 3 6 5 2 5
 3 3 8 0 9 8 7 6 1 2 0 5 0 1 3 1 4 9 0 6 9 9 5 1 2 1 2 7 4 9 6 5 6 4 6 4 9
 0 7 8 0 3 7 0 7 4 2 1 2 2 0 6 6 0 9 9 8 8 4 0 9 0 4 6 9 2 8 2 9 4 1 3 9 8
 2 4 9 8 6 9 5 7 7 2 9 7 9 6 7 4 6 9 1 8 4 4 7 1 5 4 0 2 5 9 2 4 4 4 3 1 5
 9 3 6 2 0 4 5 3 8 6 6 0 1 5 6 8 5 4 9 7 0 8 5 5 2 8 1 5 1 8 1 5 9 3 3 3 6
 0 5 0 5 2 4 6 2 6 6 0 3 9 0 2 4 9 9 2 4 9 4 1 3 1 8 9 3 2 8 6 5 0 5 3 5 7
 2 1 6 6 3 0 0 6 8 9 5 5 0 3 9 3 9 2 1 8 5 9 9 1 9 4 7 0 5 8 4 8 8 5 2 3 6
 1 4 9 2 1 0 4 5 3 7 7 8 6 8 5 8 8 7 2 7 7 5 8 7 8 6 7 2 6 7 5 4 2 2 1 0 1
 3 9 6 9 7 5 2 3 6 6 2 6 7 5 1 3 3 8 1 7 2 6 0 6 2 7 6 7 3 0 4 4 0 4 3 5 4
 7 6 6 3 3 3 6 1 4 2 6 5 4 8 7 5 6 3 2 8 7 8 9 6 5 3 2 8 4 5 0 7

In [9]:
# evaluating accuracy
accuracy = accuracy_score(y_true=y_te, y_pred=pred_X)
print('gcForest accuracy : {}'.format(accuracy))

gcForest accuracy : 0.9847009735744089


<h2>Using mg-scanning and cascade_forest Sperately</h2>
<p>As the Multi-Grain scanning and the cascade forest modules are quite independent it is possible to use them seperately.<br>
If a target `y` is given the code automaticcaly use it for training otherwise it recalls the last trained Random Forests to slice the data.</p>

In [10]:
gcf = gcForest(shape_1X=[8,8], window=[5], min_samples=7)
X_tr_mgs = gcf.mg_scanning(X_tr, y_tr)

Slicing Images...
Training MGS Random Forests...


In [11]:
X_te_mgs = gcf.mg_scanning(X_te)

Slicing Images...


<p>It is now possible to use the mg_scanning output as input for cascade forests using different parameters. Note that the cascade forest module does not directly return predictions but probability predictions from each Random Forest in the last layer of the cascade. Hence the need to first take the mean of the output and then find the max.</p>

In [15]:
gcf = gcForest(tolerance=0.0, min_samples=7)
_ = gcf.cascade_forest(X_tr_mgs, y_tr)

Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.9953703703703703
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.9953703703703703


In [23]:
pred_proba = gcf.cascade_forest(X_te_mgs)
tmp = np.mean(pred_proba, axis=0)
preds = np.argmax(tmp, axis=1)
accuracy_score(y_true=y_te, y_pred=preds)

0.97774687065368571

In [24]:
gcf = gcForest(tolerance=0.0, min_samples=20)
_ = gcf.cascade_forest(X_tr_mgs, y_tr)

Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.9675925925925926
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.9722222222222222
Adding/Training Layer, n_layer=3
Layer validation accuracy = 0.9722222222222222


In [25]:
pred_proba = gcf.cascade_forest(X_te_mgs)
tmp = np.mean(pred_proba, axis=0)
preds = np.argmax(tmp, axis=1)
accuracy_score(y_true=y_te, y_pred=preds)

0.98052851182197498